In [3]:
import pandas as pd

# Load the uploaded CSV file to inspect its structure
file_path = '../Datasets/PTSD_data.csv'
ptsd_data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
ptsd_data.head()

,Unnamed: 0,IQ,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,AB.A.delta.f.F4,AB.A.delta.g.F8,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,99.0,Posttraumatic stress disorder,17.603385,17.243334,11.729942,15.381709,18.307108,28.369365,11.764645,...,94.727123,73.994841,75.159075,72.479599,81.756815,74.578657,78.980627,60.508215,65.113378,69.238767
1,2,120.0,Posttraumatic stress disorder,21.714048,19.579805,18.522271,26.820075,25.429025,27.345290,16.839109,...,83.821476,61.617403,63.964423,73.096645,74.161249,50.432291,77.440228,35.794362,70.592405,48.283439
2,3,116.0,Posttraumatic stress disorder,13.371076,14.028142,16.901291,21.767857,18.227411,16.825877,17.859026,...,70.585726,63.864062,46.127953,65.675414,62.455949,33.114945,56.163998,37.477109,72.732968,46.665464
3,4,137.0,Posttraumatic stress disorder,30.473244,13.954586,23.056715,26.177810,27.884679,24.520958,19.931770,...,87.768539,71.799769,65.845016,67.734909,78.328065,58.141474,68.113967,53.391012,71.111448,67.598506
4,5,89.0,Posttraumatic stress disorder,18.488575,19.603144,9.899157,21.724115,32.700666,24.510528,13.788542,...,70.188502,27.698816,24.181746,34.445899,47.797466,20.737129,47.210870,11.104500,50.442443,36.952529


In [4]:
from sklearn.preprocessing import MinMaxScaler

# Extract features and normalize them
features = ptsd_data.iloc[:, 3:]  # Assuming columns after the third are EEG features
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(features)

# Display the shape of the normalized features for confirmation
normalized_features.shape


/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


(104, 1141)

In [5]:
import numpy as np

# Handle NaN values by replacing them with the column mean
nan_replaced_features = np.nan_to_num(normalized_features, nan=np.nanmean(normalized_features, axis=0))

# Confirm that there are no NaN values left
np.isnan(nan_replaced_features).sum()


/var/folders/g9/s_fx5qk126515rbzxjm9x6ww0000gn/T/ipykernel_28515/2159871591.py:4: RuntimeWarning: Mean of empty slice
  nan_replaced_features = np.nan_to_num(normalized_features, nan=np.nanmean(normalized_features, axis=0))


104

In [6]:
# Identify columns with all NaN values in the original dataset
nan_columns = np.isnan(normalized_features).all(axis=0)
columns_to_drop = np.where(nan_columns)[0]

# Drop columns with all NaN values
cleaned_features = np.delete(normalized_features, columns_to_drop, axis=1)

# Confirm the new shape and absence of NaN values
cleaned_features.shape, np.isnan(cleaned_features).sum()


((104, 1140), 0)

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential

# Define the generator model
def create_generator(input_dim, output_dim):
    model = Sequential([
        layers.Dense(256, activation='relu', input_dim=input_dim),
        layers.Dense(512, activation='relu'),
        layers.Dense(1024, activation='relu'),
        layers.Dense(output_dim, activation='tanh')  # Output scaled between -1 and 1
    ])
    return model

# Define the discriminator model
def create_discriminator(input_dim):
    model = Sequential([
        layers.Dense(1024, activation='relu', input_dim=input_dim),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output probability
    ])
    return model

# Initialize generator and discriminator
input_dim = 100  # Size of the latent vector
feature_dim = cleaned_features.shape[1]  # Number of EEG features

generator = create_generator(input_dim, feature_dim)
discriminator = create_discriminator(feature_dim)

# Compile discriminator
discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002),
                      loss='binary_crossentropy', metrics=['accuracy'])

# Freeze the discriminator during generator training
discriminator.trainable = False

# GAN model (Generator + Discriminator)
gan_input = layers.Input(shape=(input_dim,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)
gan = Model(gan_input, gan_output)
gan.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), loss='binary_crossentropy')

# Summary of the models
generator.summary(), discriminator.summary(), gan.summary()

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1140)           │     1,168,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,851,252 (7.06 MB)

 Trainable params: 1,851,252 (7.06 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 1140)           │     1,851,252 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │     1,824,769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,676,021 (14.02 MB)

 Trainable params: 1,851,252 (7.06 MB)

 Non-trainable params: 1,824,769 (6.96 MB)

(None, None, None)

In [9]:
latent_dim = 100 
batch_size = 16  # Define batch size
real_labels = np.ones((batch_size, 1))
fake_labels = np.zeros((batch_size, 1))

# Training loop with corrected variables
for epoch in range(1000):  # Set to 1000 epochs for training
    # Select a random batch of real data
    idx = np.random.randint(0, cleaned_features.shape[0], batch_size)
    real_data = cleaned_features[idx]

    # Generate a batch of fake data
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_data = generator.predict(noise)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator via the GAN
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, real_labels)

    # Print progress every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch+1}/1000 | D Loss: {d_loss[0]:.4f}, D Acc: {d_loss[1]:.4f}, G Loss: {g_loss:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'